In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL_Merchant_Linking")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/14 17:54:36 WARN Utils: Your hostname, LAPTOP-VAB0S7AL resolves to a loopback address: 127.0.1.1; using 172.31.190.2 instead (on interface eth0)
22/09/14 17:54:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/14 17:54:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
master = spark.read.parquet('../../../data/curated/process_data.parquet')

In [3]:
master.count()

7817432

In [4]:
#DATA_PATH = '../data/tables/external_datasets/'
DATA_PATH = '../../../data/tables/'
MERCHANT_PATH = 'merchant_fraud_probability.csv'


def read_merchant_data():
    merchant_fraud = spark.read.option("header", True) \
        .csv(DATA_PATH + MERCHANT_PATH)
    return merchant_fraud

In [5]:
fmerchant_sdf = read_merchant_data()

In [6]:
fmerchant_sdf.show()

+------------+--------------+------------------+
|merchant_abn|order_datetime| fraud_probability|
+------------+--------------+------------------+
| 19492220327|    2021-11-28|44.403658647495355|
| 31334588839|    2021-10-02| 42.75530083865367|
| 19492220327|    2021-12-22|38.867790051131095|
| 82999039227|    2021-12-19|  94.1347004808891|
| 90918180829|    2021-09-02| 43.32551731714902|
| 31334588839|    2021-12-26| 38.36165958070444|
| 23686790459|    2021-12-10|  79.4543441508535|
| 14827550074|    2021-11-26| 46.45775596795885|
| 31334588839|    2021-11-26| 36.20971272078342|
| 19492220327|    2021-12-18|33.819672154331755|
| 31334588839|    2021-11-29|35.386213297375505|
| 14827550074|    2021-12-05| 43.85519494291279|
| 19492220327|    2021-11-18|32.193139919494016|
| 93260930990|    2021-11-30| 37.87197154172081|
| 90918180829|    2021-09-16| 36.62001350882694|
| 83199298021|    2022-02-27|26.025158824861773|
| 83199298021|    2022-02-17| 25.77998392496447|
| 94311056026|    20

In [7]:
fmerchant_sdf.printSchema()

root
 |-- merchant_abn: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: string (nullable = true)



In [9]:
#master = master.withColumn("rate", F.col("rate").cast("double"))





In [10]:
fmerchant_sdf.printSchema()

root
 |-- merchant_abn: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: string (nullable = true)



In [11]:
fmerchant_sdf.show()

+------------+--------------+------------------+
|merchant_abn|order_datetime| fraud_probability|
+------------+--------------+------------------+
| 19492220327|    2021-11-28|44.403658647495355|
| 31334588839|    2021-10-02| 42.75530083865367|
| 19492220327|    2021-12-22|38.867790051131095|
| 82999039227|    2021-12-19|  94.1347004808891|
| 90918180829|    2021-09-02| 43.32551731714902|
| 31334588839|    2021-12-26| 38.36165958070444|
| 23686790459|    2021-12-10|  79.4543441508535|
| 14827550074|    2021-11-26| 46.45775596795885|
| 31334588839|    2021-11-26| 36.20971272078342|
| 19492220327|    2021-12-18|33.819672154331755|
| 31334588839|    2021-11-29|35.386213297375505|
| 14827550074|    2021-12-05| 43.85519494291279|
| 19492220327|    2021-11-18|32.193139919494016|
| 93260930990|    2021-11-30| 37.87197154172081|
| 90918180829|    2021-09-16| 36.62001350882694|
| 83199298021|    2022-02-27|26.025158824861773|
| 83199298021|    2022-02-17| 25.77998392496447|
| 94311056026|    20

In [12]:
def cast_to_types(merchant_sdf):
    merchant_sdf = merchant_sdf.withColumn("order_datetime_m", F.to_date("order_datetime")) \
        .withColumn('fraud_probability', merchant_sdf['fraud_probability'].cast('double') / 100) \
        .drop('order_datetime') \
        .withColumnRenamed('merchant_abn', 'merchant_abn_m')

    return merchant_sdf

In [13]:
def get_merchant_data():
    fm_sdf = read_merchant_data()
    fm_sdf = cast_to_types(fm_sdf)
    return fm_sdf

In [46]:
master.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- rate: double (nullable = true)
 |-- category: string (nullable = true)
 |-- estimated_region_population_2021_sum: long (nullable = true)
 |-- persons_earners_2018-19_sum: double (nullable = true)
 |-- mean_earnings_2018-19_avg: double (nullable = true)
 |-- sum_earnings_2018-19_sum: double (nullable = true)
 |-- median_earnings_2018-19_avg: double (nullable = true)
 |-- med_age_earners_2018-19_avg: double (nullable = true)



In [47]:
master.show(1, vertical=True)

-RECORD 0----------------------------------------------------
 merchant_abn                         | 38700038932          
 consumer_id                          | 409073               
 user_id                              | 1124                 
 dollar_value                         | 2155.27              
 order_id                             | 35059fb9-44b0-446... 
 order_datetime                       | 2021-11-25           
 state                                | NSW                  
 postcode                             | 1020                 
 gender                               | Male                 
 merchant_name                        | Etiam Bibendum In... 
 tag                                  | tent and awning s... 
 revenue                              | a                    
 rate                                 | 6.31                 
 category                             | retail_and_wholes... 
 estimated_region_population_2021_sum | 31499                
 persons

In [17]:
# join based on abn & datetime

fm_sdf = get_merchant_data()
condition = [fm_sdf['order_datetime_m'] == master['order_datetime'], fm_sdf['merchant_abn_m'] == master['merchant_abn']]
master_new = master.join(fm_sdf, condition, 'left')


In [18]:
master_new.show()

+------------+-----------+-------+------------+--------------------+--------------+-----+--------+-----------+--------------------+--------------------+-------+----+--------------------+------------------------------------+---------------------------+-------------------------+------------------------+---------------------------+---------------------------+--------------+-----------------+----------------+
|merchant_abn|consumer_id|user_id|dollar_value|            order_id|order_datetime|state|postcode|     gender|       merchant_name|                 tag|revenue|rate|            category|estimated_region_population_2021_sum|persons_earners_2018-19_sum|mean_earnings_2018-19_avg|sum_earnings_2018-19_sum|median_earnings_2018-19_avg|med_age_earners_2018-19_avg|merchant_abn_m|fraud_probability|order_datetime_m|
+------------+-----------+-------+------------+--------------------+--------------+-----+--------+-----------+--------------------+--------------------+-------+----+-----------------

In [20]:
master_new.select('fraud_probability').show()

+-----------------+
|fraud_probability|
+-----------------+
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
+-----------------+
only showing top 20 rows



In [21]:
master_new.count()

7817432

In [22]:
master_new.select('fraud_probability').na.drop().count()

4003

In [23]:
master_new.select('fraud_probability').na.drop().show()

+-------------------+
|  fraud_probability|
+-------------------+
| 0.2910241891640991|
|0.29539473036230746|
|0.29520113582407653|
|0.28337370196230055|
| 0.2908815811960701|
| 0.2900590700983104|
| 0.2758235777292768|
|0.28583030842180734|
|0.29520113582407653|
| 0.3787197154172081|
| 0.2908815811960701|
| 0.3057903215900633|
|0.31343502316867783|
|0.28956947892226464|
| 0.3057903215900633|
| 0.2900590700983104|
|0.29555244690425014|
|0.28504479048104586|
|0.28504479048104586|
|0.28504479048104586|
+-------------------+
only showing top 20 rows



In [ ]:
# only 4003 entries that arent null